In [76]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import geopandas
import cartopy.feature as cfeature
from datetime import datetime, timedelta
from cartopy import crs as ccrs
from shapely import geometry
from asmclib import geohotspot

In [64]:
ground_truth_df = geopandas.read_file('./verification/jan_jul_2019_groundtruth.geojson')

In [65]:
ground_truth_df['lat'] = ground_truth_df.geometry.y
ground_truth_df['lon'] = ground_truth_df.geometry.x

In [66]:
ground_truth_df.loc[ground_truth_df['datestamp'].str.contains('NULL'), 'datestamp'] 

Series([], Name: datestamp, dtype: object)

In [67]:
ground_truth_df.loc[23, 'datestamp'] = '25/7/2019'

In [68]:
ground_truth_df.loc[24, 'datestamp'] = '30/7/2019'

In [69]:
ground_truth_df.loc[ground_truth_df['datestamp'].str.contains('-'), 'datestamp'] 

8       5-20/7/2019
9       5-20/7/2019
10      5-20/7/2019
11      5-20/7/2019
12      5-25/7/2019
13      5-25/7/2019
14      5-25/7/2019
15      5-25/7/2019
16      5-25/7/2019
25     25-30/7/2019
26     25-30/7/2019
27      5-20/7/2019
28      5-20/7/2019
29      5-20/7/2019
30      5-20/7/2019
31      5-20/7/2019
33      5-20/7/2019
34      5-25/7/2019
36      5-25/7/2019
38      5-25/7/2019
39      5-25/7/2019
40      5-25/7/2019
41      5-25/7/2019
42      5-25/7/2019
43      5-25/7/2019
44      5-25/7/2019
45      5-25/7/2019
46      5-25/7/2019
49      5-25/7/2019
50      5-25/7/2019
           ...     
308     1-31/7/2019
309     1-31/7/2019
310     1-31/7/2019
311     1-31/7/2019
312     1-31/7/2019
313     1-31/7/2019
314     1-31/7/2019
315     1-31/7/2019
316     1-31/7/2019
318     1-31/7/2019
319     1-31/7/2019
321     1-31/7/2019
322     1-31/7/2019
323    20-31/7/2019
324    20-31/7/2019
325    20-31/7/2019
326    20-31/7/2019
327    20-31/7/2019
329     1-31/7/2019


In [70]:
ground_truth_df['startdate'] = '0'
ground_truth_df['enddate'] = ground_truth_df['datestamp']

for index, row in ground_truth_df.iterrows():
    if len(row['datestamp'].split('-')) > 1:
        ground_truth_df.loc[index, 'startdate'] = row['datestamp'].split('-')[0]
        ground_truth_df.loc[index, 'enddate'] = row['datestamp'].split('-')[1]

In [71]:
ground_truth_df.loc[ground_truth_df['enddate'] == '30/72019', 'datestamp'] = '28-30/7/2019'
ground_truth_df.loc[ground_truth_df['enddate'] == '30/72019', 'enddate'] = '30/7/2019'

In [77]:
#exception handling
ground_truth_df.loc[267, 'startdate'] = '27'
ground_truth_df.loc[267]
ground_truth_df['end_date'] = np.nan

In [81]:
for index, row in ground_truth_df.iterrows():
    end_day = int(row['enddate'].split('/')[0])
    end_month = int(row['enddate'].split('/')[1])
    end_year = int(row['enddate'].split('/')[2])
    start_day = int(row['startdate'])
    ground_truth_df.loc[index, 'end_date'] = datetime(end_year, end_month, end_day)
        
    if start_day > end_day:
        start_month = end_month - 1
    else:
        start_month = end_month
                
    if row['startdate'] == '0':
        ground_truth_df.loc[index, 'start_date'] = datetime(end_year, end_month, end_day)
    else:
        ground_truth_df.loc[index, 'start_date'] = datetime(end_year, start_month, start_day)

In [82]:
ground_truth_df['period'] = ground_truth_df['end_date'] - ground_truth_df['start_date']

In [83]:
ground_truth_gjson = ground_truth_df.copy()
ground_truth_gjson

,datestamp,lat,lon,period,geometry,startdate,enddate,end_date,start_date
0,20/7/2019,-0.126831,101.696714,0 days,POINT (101.69671 -0.12683),0,20/7/2019,2019-07-20 00:00:00,2019-07-20 00:00:00
1,20/7/2019,-0.137187,101.699419,0 days,POINT (101.69942 -0.13719),0,20/7/2019,2019-07-20 00:00:00,2019-07-20 00:00:00
2,20/7/2019,-0.124976,101.709003,0 days,POINT (101.70900 -0.12498),0,20/7/2019,2019-07-20 00:00:00,2019-07-20 00:00:00
3,25/7/2019,-0.127913,101.708152,0 days,POINT (101.70815 -0.12791),0,25/7/2019,2019-07-25 00:00:00,2019-07-25 00:00:00
4,25/7/2019,-0.132241,101.698337,0 days,POINT (101.69834 -0.13224),0,25/7/2019,2019-07-25 00:00:00,2019-07-25 00:00:00
5,25/7/2019,-0.163310,101.704597,0 days,POINT (101.70460 -0.16331),0,25/7/2019,2019-07-25 00:00:00,2019-07-25 00:00:00
6,20/7/2019,-0.172894,101.705061,0 days,POINT (101.70506 -0.17289),0,20/7/2019,2019-07-20 00:00:00,2019-07-20 00:00:00
7,5/7/2019,-0.140588,101.720364,0 days,POINT (101.72036 -0.14059),0,5/7/2019,2019-07-05 00:00:00,2019-07-05 00:00:00
8,5-20/7/2019,-0.155582,101.846265,15 days,POINT (101.84626 -0.15558),5,20/7/2019,2019-07-20 00:00:00,2019-07-05 00:00:00
9,5-20/7/2019,-0.151408,101.844333,15 days,POINT (101.84433 -0.15141),5,20/7/2019,2019-07-20 00:00:00,2019-07-05 00:00:00


In [86]:
ground_truth_gjson = ground_truth_gjson.to_crs({'init': 'epsg:3857'})
ground_truth_gjson.drop(columns=['startdate', 'enddate'], inplace=True)

KeyError: "['startdate' 'enddate'] not found in axis"

In [91]:
ground_truth_gjson.to_csv('./verification/ground_truth_2019.csv')